## AFLOW-ML API


AFLOW-ML provides an open RESTful API to directly access continuously updated ML models, which can be integrated into any workflow to retrieve predictions of electronic, thermal and mechanical properties. These types of interconnected cloud-based applications have the potential to further accelerate the adoption of machine learning methods into materials development.


### API structure


The AFLOW-ML API is structured around a REpresentational State Transfer (REST) architecture, which allows resources to be accessed using HTTP request methods. Each resource is located at an **endpoint**, which is identified by a URL comprised of descriptive nouns.


Resources within the API are represented in JavaScript Object Notation (JSON), and are referred to as **objects**. Once at an endpoint, the user must specify how to interact with the object. This is referred to as an **action**, and is an HTTP request method. The API supports the two most common HTTP request methods, ```GET``` and ```POST```, where ```GET``` fetches an object and ```POST``` sends user defined data to the server. Therefore, users interact with the API by performing actions (```GET```, ```POST```) on endpoints (URLs) to retrieve objects (resources).

#### Endpoints

All endpoints are located at the base URL http://aflow.org/API/aflow-ml/v1.0/ and are organized by the model and the returned object.

| Action | Endpoint               |        Object          |
| ----  |:-----------------------:| ----------------------:|
| POST  | /plmf/prediction        | Task                   |
| POST  | /mfd/prediction         | Task                   |
| GET   | /prediction/result/{id} |   Status or prediction |

In general, API usage involves uploading a material structure to a POST endpoint, {model}/prediction, and retrieving a prediction object from a GET endpoint, /prediction/result/{id}, as shown in the flowchart below:

![](./images/aflow-ml_workflow.png)

```POST``` endpoints are responsible for the submission of a material structure for a prediction. In their request body, the file keyword is required. It must contain a string representation of the material’s crystal structure, in POSCAR 5 format.

Upon receiving a request, the response body returns a **task object** containing information about the submitted structure, which has the following format:

```JavaScript
{
     "id": String,
     "model": String,
     "results_endpoint": String
}
```

When a material is posted to the API, a prediction task is created and added to a queue. Each task is assigned an identifier, the id keyword, used to fetch the prediction object at the endpoint referenced in the results_endpoint keyword. This endpoint, /prediction/result/{id}, supports the ```GET``` method and requires the id as a path parameter. Depending on the status of the prediction task, the response body returns a status object or prediction object. 

```JavaScript
// status object 
{
     "status": String,
     "description": String
}
```

Prediction objects are returned depending on the targeted model. Currently we support two models: molar fraction descriptor (mfd) and property-labeled materials fragments (plmf).

### ML models

#### property-labeled materials fragments (plmf)
The plmf model represents each crystal structure as a colored graph, where the atomic vertices are decorated by the reference properties of the corresponding elemental species. Topological neighbors are determined using a Voronoi tessellation, and these nodes are connected to form the graph. The final feature vector for the ML model is obtained by partitioning the full graph into smaller subgraphs, termed fragments in analogy with the fragment-based descriptors used in cheminformatics. All plmf models are built with the Gradient Boosting Decision Tree (GBDT) method. Models for electronic and thermo-mechanical properties were trained on 26,674 and 2,829 materials entries from the AFLOW repository, respectively. All models are validated through Y-randomization (label scrambling) and fivefold cross validation, with coefficient of determination (r^2) values in excess of 0.9 for most quantities, while the mean average errors are 0.035 eV (electronic band gap), 8.68 GPa (bulk modulus), 10.6 GPa (shear modulus), 35.9 K (Debye temperature), 0.05 K_b/atom (heat capacity at constant pressure) and 0.04 K_b/atom (heat capacity at constant volume), and the root mean square errors are 0.51 eV (electronic band gap), 14.3 GPa (bulk modulus), 18.4 GPa (shear modulus), 57.0 K (Debye temperature), 0.09 K_b/atom (heat capacity at constant pressure) and 0.07 K_b/atom (heat capacity at constant volume). Further details on the model training and validation can be found here: https://www.nature.com/articles/ncomms15679


The plmf prediction object has the following structure:

```JavaScript
// plmf prediction object 
{
     "status": String,
     "description": String,
     "model": String,
     "citation": String,
     "ml_egap_type": String,
     "ml_egap": Number,
     "ml_energy_per_atom": Number,
     "ml_ael_bulk_modulus_vrh": Number,
     "ml_ael_shear_modulus_vrh": Number,
     "ml_agl_debye": Number,
     "ml_agl_heat_capacity_Cp_300K": Number,
     "ml_agl_heat_capacity_Cv_300K": Number,
     "ml_agl_heat_capacity_Cp_300K_per_atom": Number,
     "ml_agl_heat_capacity_Cv_300K_per_atom": Number,
     "ml_agl_thermal_conductivity_300K": Number,
     "ml_agl_thermal_expansion_300K": Number
}
```

#### molar fraction descriptor (mfd)
The mfd model predicts the material properties based on the chemical formula only: the vector of descriptors has 87 components, each component being the mole fraction of element in the compound ( is H, is He, etc.). The model is built with nonlinear support vector machines and a radial basis function kernel. The model is trained using a data set of 292 randomly selected compounds of the ICSD for which the vibrational properties are computed with DFT calculations. Pearson and Spearman correlations for k-fold cross-validation () are in excess of 0.9 for all predicted properties, while the mean average errors are 13.2 meV/atom (vibrational free energy) and 0.037 meV/(atom K) (vibrational entropy) and the root mean square errors are 18.8 meV/atom (vibrational free energy) and 0.052 eV/(atom K) (vibrational entropy). Further details on the model training and validation can be found in here: https://pubs.acs.org/doi/abs/10.1021/acs.chemmater.7b00789

The mfd prediction object takes the following structure:

```JavaScript
// mfd prediction object 
{
     "status": String,
     "description": String,
     "model": String,
     "citation": String,
     "ml_Cv": Number,
     "ml_Fvib": Number,
     "ml_Svib": Number
}
```




## Using the API

The process to retrieve a prediction is as follows: First, the contents of a POSCAR 5 file (i.e. the geometry input file format for version 5 of VASP), titled test.poscar, are uploaded to the submission endpoint. This can be achieved by using an HTTP library such as Requests (Python), URLSession (iOS SDK), HttpURLConnection (Java), Fetch (JavaScript) or using a command line tool such as Wget or cURL.


### cURL example

For this example the UNIX utility cURL will be used. If cURL is not available on your machine, you can use the python example below. 

The contents of a POSCAR are posted to the submission endpoint as follows:

In [ ]:
!curl http://aflow.org/API/aflow-ml/v1.0/plmf/prediction --data-urlencode file@test.poscar

the ```--data-urlencode``` flag handles encoding the contents of the poscar named ```test.poscar``` found in the current directory. From the JSON response above we will use the task id to poll the results endpoint.

In [ ]:
!curl http://aflow.org/API/aflow-ml/v1.0/prediction/result/<insert_your_task_id_here>

### Python example

For this example the Python 3 Requests library will be used. 

The contents of a POSCAR are posted to the submission endpoint as follows:

In [ ]:
#!/usr/bin/python3
import json, sys, os
from time import sleep
from urllib.parse import urlencode
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import HTTPError

SERVER="http://aflow.org"
API="/API/aflow-ml/v1.0"
MODEL="plmf"

poscar=open('test.poscar', 'r').read()
encoded_data = urlencode({'file': poscar,}).encode('utf-8')

url = SERVER + API + "/" + MODEL + "/prediction"
request_task = Request(url, encoded_data)
task = urlopen(request_task).read()
task_json = json.loads(task.decode('utf-8'))
results_endpoint = task_json["results_endpoint"]
results_url = SERVER + API + results_endpoint

print(results_url)

The prediction can be retrieved by pasting the results_url string provided into the <results_url> placeholder in the code below:

In [ ]:
#!/usr/bin/python3
import json, sys, os
from time import sleep
from urllib.parse import urlencode
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import HTTPError

SERVER="http://aflow.org"
API="/API/aflow-ml/v1.0"
MODEL="plmf"

results_url = <results_url>

request_results = Request(results_url)
results = urlopen(request_results).read()
results_json = json.loads(results)

if results_json["status"] == 'PENDING':
    print("AFLOW-ML prediction pending")
elif results_json["status"] == 'STARTED':
    print("AFLOW-ML prediction started")
elif results_json["status"] == 'FAILURE':
    print("Error: AFLOW-ML prediction failure")
elif results_json["status"] == 'SUCCESS':
    print("AFLOW-ML successful prediction")
    print(results_json)

The POSCAR submission and results retrieval steps can be combined into a single code as shown below:

In [ ]:
#!/usr/bin/python3
import json, sys, os
from time import sleep
from urllib.parse import urlencode
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import HTTPError

SERVER="http://aflow.org"
API="/API/aflow-ml/v1.0"
MODEL="plmf"

poscar=open('test.poscar', 'r').read()
encoded_data = urlencode({'file': poscar,}).encode('utf-8')

url = SERVER + API + "/" + MODEL + "/prediction"
request_task = Request(url, encoded_data)
task = urlopen(request_task).read()
task_json = json.loads(task.decode('utf-8'))
results_endpoint = task_json["results_endpoint"]
results_url = SERVER + API + results_endpoint

incomplete = True
while incomplete:
    request_results = Request(results_url)
    results = urlopen(request_results).read()
    results_json = json.loads(results)
    if results_json["status"] == 'PENDING':
        sleep(10)
        continue
    elif results_json["status"] == 'STARTED':
        sleep(10)
        continue
    elif results_json["status"] == 'FAILURE':
        print("Error: AFLOW-ML prediction failure")
        incomplete = False
    elif results_json["status"] == 'SUCCESS':
        print("AFLOW-ML successful prediction")
        print(results_json)
        incomplete = False

## Exercises

1. Retrieve one of the POSCAR files that you generated during the AFLOW Prototypes tutorial. Convert to the VASP 5 POSCAR format if necessary, by adding an additional line with the list of elements in alphabetical order, right underneath the lattice vectors. For example, this VASP 4 format POSCAR:

would become:

Copy the submission/retrieval code into a new cell below, modify it to point to this new POSCAR, and run it to obtain the predictions for this material.

2. Modify the submission/retrieval code to print out only the prediction for the bulk modulus, instead of the entire JSON file.

3. Modify the submission/retrieval code to submit a prediction request for the mfd model instead of the plmf model.

## AFLOW-ML Python Client

AFLOW-ML is a python client for the AFLOW-ML API. It provides a clean interface which simplifies fetching predictions. Furthermore, this allows users a means for clean integration into existing codebases. 

In Addition, the module installs a command line interface (CLI), allowing for on the fly predictions -- no script required.

## Installation

The Python client is built without any dependencies. To install, fetch the tar.gz file from http://aflow.org/src/aflow-ml/. Upon untarring, cd into the new directory and run the following command:

```
pip3 install .
```

This will install the AFLOW-ML client to your global Python installation. If you do not want to install it globally, you can also create a virtual environment. 

In [ ]:
from pprint import pprint
from aflowml.client import AFLOWmlAPI

with open('test.poscar', 'r') as input_file:
    ml = AFLOWmlAPI()
    data = ml.get_prediction(input_file.read(), 'plmf')
    pprint(data)

## CLI

To install the CLI, the Python client must first be installed. The command below will install it from the notebook (assuming the contents of the library are in the same directory).

If this is not the case open the terminal change to the AFLOW-ML client directory and run:

```pip3 install .```

### List of options

In [ ]:
!aflow-ml -h

### Fetching a prediction

In [ ]:
!aflow-ml test.poscar -m plmf

### Specifying what fields to show

In [ ]:
!aflow-ml test.poscar -m plmf --fields=ml_egap,ml_egap_type

### Saving output
Saving is done via the ```-s``` flag. By default it will save as a text file named ```prediction.txt```. The ```-v``` is added to enable verbose mode.

In [ ]:
!aflow-ml test.poscar -m plmf -s -v

In [ ]:
!cat prediction.txt

### Saving to a JSON

Here we will save the output to a JSON file via the ```--format``` argument and specify the filename using the ```--outfile``` argument.

In [ ]:
!aflow-ml test.poscar -m plmf -s -v --format=json --outfile=test.json

In [ ]:
"""
Quick script to pretty print the JSON.
"""
import json
from pprint import pprint

with open('test.json', 'r') as json_pred:
    content = json.load(json_pred)
    pprint(content)

## Exercise

In this example we will predict the shear and bulk moduli for MoTi systems using the AFLOW-ML API and the AFLOW data API.


Using the AFLOW REST API fetch the poscars and auid for MoTi and save them to a file called ```entries_data.json```.

In [ ]:
import json
from urllib.request import urlopen
from urllib.request import Request

BASE_URL = 'http://aflowlib.duke.edu/AFLOWDATA/LIB2_RAW/Mo_pvTi_sv'

"""
    Your code here
"""

Now using the resultant json file ```entries_data.json``` write a script to use the AFLOW-ML client to fetch predictions, save them to an array and print the output.

**Some helpful things:**

You will want to import the json module. In addition, the pretty print module will also be handy to print the results.

```python
import json
from pprint import pprint
```

In case you forgot or are new to Python the way to open a file is as follows:

```python
with open('file_path', 'r') as my_file:
    # logic to handle files here
```

Here ```my_file``` is the file handler you can use in the with statement to access it's contents. The ```'r'``` stands for read while other options include ```'w'``` for write and ```'a'``` for append.

In [ ]:
import json
from pprint import pprint
from aflowml.client import AFLOWmlAPI

with open('entries_data.json', 'r') as json_file:
    entries_data = json.load(json_file)
    """
        Your code here
    """
